In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from smc_utils import drop_identifiers

In [2]:
df_smc = pd.read_excel(os.path.join("..","Data","SMC_DATA_clean.xlsx"))
df_goals = pd.read_csv(os.path.join("..","Coding Scheme","Goals Final Codes","goals_final_participant_level.csv"))
df_behav = pd.read_csv(os.path.join("..","Coding Scheme","Behaviors Final Codes","behaviors_final_participant_level.csv"))

In [3]:
df_smc = drop_identifiers(df_smc)
df_goals = drop_identifiers(df_goals)
df_behav = drop_identifiers(df_behav)

In [4]:
# rename past codes
df_goals.rename(columns={'Past':'Past_g'}, inplace=True)
df_behav.rename(columns={'Past':'Past_b'}, inplace=True)

In [5]:
# keep only initial survey data
df_smc = df_smc[df_smc['SURVEY'] == 0]

In [6]:
# join goals and behaviors
df_smc = df_smc.merge(df_goals, on='ParticipantID')
df_smc = df_smc.merge(df_behav, on='ParticipantID')
df_smc.head()

,UniqueID,ParticipantID,ParticipantRecordCount,SEMESTER,SURVEY,Timestamp,STATE_behaviors,STATE_goals,COVID_residence,COVID_social_distancing,...,Hedge,Past_g,SAT,DIS,OU,OI,OP,PSI,PCC,Past_b
0,a9dbc45ece05459c91b7073f94165d9d,5710075738,5,Fall 2020,0,2020-09-20 17:17:09.790,"I generally use Reddit, Snapchat, Twitter, Ins...",I’d like to see myself distance a bit from soc...,In Boulder,4.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,ed0240e0259b415b92afcc633a9a017f,2563157200,5,Fall 2020,0,2020-09-20 19:11:21.316,Most used platforms:\n- instagram\n- snapchat\...,1. I would like to limit my use of snapchat an...,In Boulder,4.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,b28032790cb44cc2bc6923ab902392be,2122687351,5,Fall 2020,0,2020-09-19 13:44:59.325,"\n- I probably check each one 5 times a day, d...",- I would like to not always go on social med...,In Boulder,4.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
3,f6e5d609f65b4900aa35cee56e635c27,480385374,5,Fall 2020,0,2020-09-20 12:50:12.303,I use reddit and occasionally YouTube. I used ...,I think my behaviors are relatively healthy no...,At my family home,4.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
4,beb6ed14af234e0e93f96c7a4993fed0,5859634604,5,Fall 2020,0,2020-09-21 21:11:57.266,"I use youtube, amazon video, netflix all for c...",I am pretty happy with where I am right now. I...,At my family home,5.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0


In [7]:
survey_cols = [c for c in df_smc.columns if c.endswith('_total')]
goals_cols = [c for c in df_goals if not c.startswith('Participant')]
behav_cols = [c for c in df_behav if not c.startswith('Participant')]

In [8]:
survey_cols = [
    'BSMAS_total',
    'SoNA_total',
    'ADTS_ANX_total',
    'PSS_total',
    'ChQ_total',
    'SWLS_total',
    'ADTS_N_total',
    'SoPA_total',
    'ADTS_P_total'
]

survey_valid_sems = {
    'SoNA_total': ['Spring 2023'],
    'ADTS_ANX_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022', 'Fall 2022'],
    'PSS_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022'],
    'ChQ_total': ['Spring 2023'],
    'SWLS_total': ['Fall 2020', 'Spring 2021'],
    'ADTS_N_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022', 'Fall 2022'],
    'ADTS_P_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022', 'Fall 2022'],
    'SoPA_total': ['Spring 2023']
}

In [9]:
df_smc[survey_cols + goals_cols + behav_cols].corr().iloc[:len(survey_cols),len(survey_cols):]

,OI Change,OP Change,PSI Change,PCC Change,RWBC,DBC,S_DBC,Mindful,Hedge,Past_g,SAT,DIS,OU,OI,OP,PSI,PCC,Past_b
BSMAS_total,-0.003478,-0.088205,0.049465,-0.042652,0.088705,0.095281,0.072145,0.055913,0.039358,0.026807,-0.043991,0.025175,0.185147,-0.089688,-0.091661,-0.025090,-0.074535,-0.002204
SoNA_total,-0.079956,-0.084079,0.085744,0.033518,-0.017040,-0.060206,-0.031958,0.072422,0.088066,-0.245669,-0.209713,-0.059619,0.074669,-0.110363,-0.236455,0.023435,0.089329,-0.044827
ADTS_ANX_total,0.075441,0.015880,0.059239,-0.041264,0.124851,0.071473,0.037038,0.022227,0.094519,-0.051348,-0.027063,0.052901,0.146281,0.027382,0.010837,0.090302,0.037115,0.006685
PSS_total,-0.018222,-0.088039,0.092016,-0.029448,-0.038068,-0.115798,-0.112199,-0.001415,0.004041,-0.094625,-0.108709,-0.102622,0.011216,-0.037537,-0.095056,-0.009263,-0.038603,-0.032821
ChQ_total,-0.091816,-0.037950,-0.115066,0.007056,0.136240,0.305809,0.122183,-0.222018,-0.028868,0.099564,-0.126164,0.256565,0.096607,-0.292661,-0.239561,-0.173602,0.084922,0.408789
SWLS_total,0.073734,-0.017556,0.070340,-0.059152,0.114480,0.123338,0.167848,-0.018049,-0.008885,0.014742,0.140750,0.046321,-0.028137,-0.033794,0.082246,0.103107,0.017859,0.087001
ADTS_N_total,0.004048,0.021056,-0.005151,-0.003678,0.012683,0.076590,0.084674,0.060132,-0.000548,0.057062,-0.025365,0.056126,0.085410,0.002684,0.050630,-0.059525,0.001236,0.081418
SoPA_total,-0.156331,0.171688,-0.036191,0.133151,0.142970,-0.048480,-0.183401,-0.018289,0.143825,0.053530,0.123633,0.047809,0.017566,0.016105,0.038858,0.128177,-0.141551,0.120800
ADTS_P_total,0.090289,0.032810,0.051504,-0.031869,0.045561,0.021911,0.019118,0.008556,0.131551,-0.022123,0.023099,0.028160,-0.023207,0.041994,0.078770,0.098360,0.019869,-0.038542


In [10]:
abs(df_smc[survey_cols + goals_cols + behav_cols].corr().iloc[:len(survey_cols),len(survey_cols):]) > 0.1

,OI Change,OP Change,PSI Change,PCC Change,RWBC,DBC,S_DBC,Mindful,Hedge,Past_g,SAT,DIS,OU,OI,OP,PSI,PCC,Past_b
BSMAS_total,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
SoNA_total,False,False,False,False,False,False,False,False,False,True,True,False,False,True,True,False,False,False
ADTS_ANX_total,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False
PSS_total,False,False,False,False,False,True,True,False,False,False,True,True,False,False,False,False,False,False
ChQ_total,False,False,True,False,True,True,True,True,False,False,True,True,False,True,True,True,False,True
SWLS_total,False,False,False,False,True,True,True,False,False,False,True,False,False,False,False,True,False,False
ADTS_N_total,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
SoPA_total,True,True,False,True,True,False,True,False,True,False,True,False,False,False,False,True,True,True
ADTS_P_total,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


In [11]:
# interesting finding - those who indicated a higher desire for change via ChQ tended to talk about behaviors in the past tense

## We will cluster using as much data as possible from our 6 semesters. SO, we need to estimate the following

<ol>
    <li>PSS for F22, S23</li>
    <li>ADTS_ANX/N/P for S23 </li>
    <li>SWLS for F21, S22, F22, S23</li>
    <li>SOPA, SONA, CHQ for F20, S21, F21, S22, F22</li>
</ol>

In [12]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    powerset = chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1))
    return [list(item) for item in powerset]

In [13]:
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate

def regression_iterator(data, target_survey, features, cv_folds=5, model_type='both'):
    '''
    data - all SMC data
    target - value to predict
    features - subset of candidate features to include in regression
    cv_folds - number of desired folds for cross validation
    model_type - multiple linear regressor OR random forest regressor
    
    trains regression models with each possible comination of features with
    k-fold cross validation, recording R^2, MSE, RMSE
    
    returns a dictionary of feature choices and their metrics
    '''
    survey_valid_sems = {
        'SoNA_total': ['Spring 2023'],
        'ADTS_ANX_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022', 'Fall 2022'],
        'PSS_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022'],
        'ChQ_total': ['Spring 2023'],
        'SWLS_total': ['Fall 2020', 'Spring 2021'],
        'ADTS_N_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022', 'Fall 2022'],
        'ADTS_P_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022', 'Fall 2022'],
        'SoPA_total': ['Spring 2023']
    }
    # subset data to include only valid semesters and included features
    data = data[data['SEMESTER'].isin(survey_valid_sems[target_survey])].copy()
    data = data[[target_survey] + features]
    
    # shuffle data
    #shuffle_indices = np.random.permutation(data.index)
    #data = data.iloc[shuffle_indices]
    
    # fill in nans
    nan_log = {}
    for feature in features:
        na_count = data[feature].isna().sum()
        if na_count > 0:
            na_prop = round((na_count / len(data)) * 100, 2)
            nan_log[feature] = 'imputing {} nan values ({}% of data)'.format(na_count, na_prop)
            data[feature].fillna(value=data[feature].mean(), inplace=True)
    
    # get list of feature choices for model
    feature_combos = powerset(features)
    
    # init results dict
    results = {}
    best_r2 = 0
    best_r2_combo = None
    best_rmse = np.inf
    best_rmse_combo = None
    
    # start loop
    for combo in feature_combos:
        X = data[combo]
        y = data[target_survey]
        
        # model selection
        assert model_type in ['mlr', 'rfr']
        if model_type == 'mlr':
            model = LinearRegression()
        elif model_type == 'rfr':
            model = RandomForestRegressor()
        
        # cross validation
        scores = cross_validate(model, X, y, cv=cv_folds,
                                scoring=('r2', 'neg_root_mean_squared_error'),
                                return_train_score=False)
        
        # store results
        results[str(combo)] = {}
        results[str(combo)]['r2'] = round(scores['test_r2'].mean(), 2)
        results[str(combo)]['rmse'] = round(scores['test_neg_root_mean_squared_error'].mean(), 2)
        
        if round(scores['test_r2'].mean(), 2) > best_r2:
            best_r2 = round(scores['test_r2'].mean(), 2)
            best_r2_combo = str(combo)
        
        if abs(round(scores['test_neg_root_mean_squared_error'].mean(), 2)) < best_rmse:
            best_rmse = abs(round(scores['test_neg_root_mean_squared_error'].mean(), 2))
            best_rmse_combo = str(combo)
    
    return results, nan_log, best_r2_combo, best_r2, best_rmse_combo, best_rmse

In [14]:
df_smc['SoNA_total'].isna().sum() / len(df_smc)

0.883399209486166

In [15]:
def select_candidate_features(data, target, corr_threshold=0.1):
    feature_options = data[survey_cols + goals_cols + behav_cols].corr().loc[target,:]
    return feature_options[abs(feature_options) > 0.1].sort_values(ascending=False)

In [16]:
def find_best_mlr_features(var_name, df_smc):
    features = select_candidate_features(df_smc, var_name)
    print("Candidate features:\n", str(features))
    candidate_features = features.index.values[1:].tolist()
    results, nan_log, best_r2_combo, best_r2, best_rmse_combo, best_rmse = regression_iterator(df_smc, var_name, candidate_features, model_type='mlr')
    print('\n', nan_log, '\n')
    print("\nBest R^2 combo: {}, R^2={}".format(best_r2_combo, best_r2))
    print("\nBest RMSE combo: {}, RMSE={}".format(best_rmse_combo, best_rmse))
    return results

## RESULTS


In [18]:
dvars = [
    "PSS_total",
    "ADTS_ANX_total",
    "ADTS_N_total",
    "ADTS_P_total",
    "SWLS_total",
    "SoPA_total",
    "SoNA_total",
    "ChQ_total"
]

results = {}

for var in dvars:
    print("******", var, "******")
    results[var] = find_best_mlr_features(var, df_smc)
    print("\n**************************\n")

****** PSS_total ******
Candidate features:
 PSS_total         1.000000
BSMAS_total       0.394119
ADTS_ANX_total    0.237564
DIS              -0.102622
SAT              -0.108709
S_DBC            -0.112199
DBC              -0.115798
SWLS_total       -0.470724
Name: PSS_total, dtype: float64

 {'SWLS_total': 'imputing 180 nan values (50.7% of data)'} 


Best R^2 combo: ['BSMAS_total', 'DIS', 'SWLS_total'], R^2=0.21

Best RMSE combo: ['BSMAS_total', 'DIS', 'SAT', 'DBC', 'SWLS_total'], RMSE=8.01

**************************

****** ADTS_ANX_total ******
Candidate features:
 ADTS_ANX_total    1.000000
BSMAS_total       0.436362
PSS_total         0.237564
ADTS_P_total      0.208420
OU                0.146281
RWBC              0.124851
SWLS_total       -0.134607
Name: ADTS_ANX_total, dtype: float64

 {'BSMAS_total': 'imputing 1 nan values (0.22% of data)', 'PSS_total': 'imputing 92 nan values (20.58% of data)', 'SWLS_total': 'imputing 272 nan values (60.85% of data)'} 


Best R^2 combo: ['BS

In [19]:
df_smc[['ChQ_total','SoNA_total','SoPA_total']].describe()

,ChQ_total,SoNA_total,SoPA_total
count,59.000000,59.000000,59.000000
mean,16.847458,15.779661,21.491525
std,4.084511,4.060184,3.559218
min,8.000000,7.000000,12.000000
25%,14.500000,13.000000,19.500000
50%,18.000000,15.000000,22.000000
75%,19.000000,19.000000,23.500000
max,25.000000,24.000000,29.000000


## Train models and impute data - save data with new name - SMC_DATA_wk0_mlr_imputed

In [20]:
# PSS_total: Best RMSE combo: ['BSMAS_total', 'DIS', 'SAT', 'DBC', 'SWLS_total'], RMSE=8.01
# ADTS_ANX_total: Best RMSE combo: ['BSMAS_total', 'PSS_total', 'ADTS_P_total', 'RWBC'], RMSE=2.65
# ADTS_N_total: Best RMSE combo: ['BSMAS_total', 'ADTS_P_total'], RMSE=2.75
# ADTS_P_total: Best RMSE combo: ['ADTS_ANX_total', 'SWLS_total', 'ADTS_N_total'], RMSE=1.63
# SWLS_total: Best RMSE combo: ['ADTS_P_total', 'SAT', 'PSS_total'], RMSE=5.61
# SoPA_total: Best RMSE combo: ['OP Change', 'Hedge', 'RWBC', 'PCC Change', 'OI Change', 'SoNA_total'], RMSE=3.33
# SoNA_total: Best RMSE combo: ['BSMAS_total', 'SoPA_total'], RMSE=3.41
# ChQ_total: Best RMSE combo: ['Past_b', 'BSMAS_total', 'DIS', 'OP'], RMSE=3.41

In [21]:
def train_and_predict(data, target_survey, features):
    '''
    train mlr for target_survey using features, then predict for those data points
    which are missing from semesters in the SMC data
    '''
    survey_valid_sems = {
        'SoNA_total': ['Spring 2023'],
        'ADTS_ANX_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022', 'Fall 2022'],
        'PSS_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022'],
        'ChQ_total': ['Spring 2023'],
        'SWLS_total': ['Fall 2020', 'Spring 2021'],
        'ADTS_N_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022', 'Fall 2022'],
        'ADTS_P_total': ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022', 'Fall 2022'],
        'SoPA_total': ['Spring 2023']
    }
    all_sems = ['Fall 2020', 'Spring 2021', 'Fall 2021', 'Spring 2022', 'Fall 2022', 'Spring 2023']
    
    data = data.copy()
    
    # fill in nans
    nan_log = {}
    for feature in features:
        na_count = data[feature].isna().sum()
        if na_count > 0:
            na_prop = round((na_count / len(data)) * 100, 2)
            nan_log[feature] = 'imputing {} nan values ({}% of data)'.format(na_count, na_prop)
            data[feature].fillna(value=data[feature].mean(), inplace=True)
            
            
    # subset data to include only valid semesters and included features
    data_train = data[data['SEMESTER'].isin(survey_valid_sems[target_survey])].copy()
    X_train = data_train[features]
    y_train = data_train[target_survey]
    
    data_pred = data[~data['SEMESTER'].isin(survey_valid_sems[target_survey])].copy()
    X_pred = data_pred[features]
            
    mlr = LinearRegression()
    mlr.fit(X_train, y_train)
    
    preds = mlr.predict(X_pred)
    
    return pd.Series(preds, index=X_pred.index)

In [22]:
pss_preds = train_and_predict(df_smc, 'PSS_total', ['BSMAS_total', 'DIS', 'SAT', 'DBC', 'SWLS_total'])

In [23]:
adts_anx_preds = train_and_predict(df_smc, 'ADTS_ANX_total', ['BSMAS_total', 'PSS_total', 'ADTS_P_total', 'RWBC'])

In [24]:
adts_n_preds = train_and_predict(df_smc, 'ADTS_N_total', ['BSMAS_total', 'ADTS_P_total'])

In [25]:
adts_p_preds = train_and_predict(df_smc, 'ADTS_P_total', ['ADTS_ANX_total', 'SWLS_total', 'ADTS_N_total'])

In [26]:
swls_preds = train_and_predict(df_smc, 'SWLS_total', ['ADTS_P_total', 'SAT', 'PSS_total'])

In [27]:
sopa_preds = train_and_predict(df_smc, 'SoPA_total', ['OP Change', 'Hedge', 'RWBC', 'PCC Change', 'OI Change', 'SoNA_total'])

In [28]:
sona_preds = train_and_predict(df_smc, 'SoNA_total', ['BSMAS_total', 'SoPA_total'])

In [29]:
chq_preds = train_and_predict(df_smc, 'ChQ_total', ['Past_b', 'BSMAS_total', 'DIS', 'OP'])

In [30]:
df_smc.loc[pss_preds.index,'PSS_total'] = pss_preds
df_smc.loc[adts_anx_preds.index,'ADTS_ANX_total'] = adts_anx_preds
df_smc.loc[adts_n_preds.index,'ADTS_N_total'] = adts_n_preds
df_smc.loc[adts_p_preds.index,'ADTS_P_total'] = adts_p_preds
df_smc.loc[swls_preds.index,'SWLS_total'] = swls_preds
df_smc.loc[sopa_preds.index,'SoPA_total'] = sopa_preds
df_smc.loc[sona_preds.index,'SoNA_total'] = sona_preds
df_smc.loc[chq_preds.index,'ChQ_total'] = chq_preds

In [31]:
df_smc[dvars].describe()

,PSS_total,ADTS_ANX_total,ADTS_N_total,ADTS_P_total,SWLS_total,SoPA_total,SoNA_total,ChQ_total
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,26.173461,10.056159,9.805517,7.572707,23.399575,21.765973,15.522795,17.760821
std,7.963562,2.926866,2.667368,1.596647,4.144838,1.997373,2.063639,3.122629
min,0.000000,3.000000,3.000000,2.000000,5.000000,12.000000,7.000000,8.000000
25%,22.000000,8.000000,8.000000,7.000000,22.000000,20.305461,14.440106,16.000000
50%,26.000000,10.000000,10.000000,7.572707,23.511664,21.579378,15.411832,17.650548
75%,31.000000,12.000000,12.000000,9.000000,25.000000,22.567606,16.707467,18.891373
max,46.000000,15.000000,15.000000,10.000000,35.000000,29.000000,24.000000,25.999486


In [32]:
df_smc[dvars].isna().sum()

PSS_total         0
ADTS_ANX_total    0
ADTS_N_total      0
ADTS_P_total      0
SWLS_total        0
SoPA_total        0
SoNA_total        0
ChQ_total         0
dtype: int64

In [33]:
df_smc.to_excel(os.path.join("..","Data","SMC_DATA_wk0_mlr_imputed_deidentified.xlsx"))